In [1]:
import logging
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import scarches as sca
import scvi
from scvi.model import SCVI
import scipy.sparse
import anndata
import os
from scib.metrics import metrics
from lataq_reproduce.exp_dict import EXPERIMENT_INFO
from lataq_reproduce.utils import label_encoder

# 自定义 remove_sparsity
def custom_remove_sparsity(adata):
    if scipy.sparse.issparse(adata.X):
        X_dense = adata.X.toarray().astype(np.int64)
        return sc.AnnData(X=X_dense, obs=adata.obs.copy(deep=True), var=adata.var.copy(deep=True))
    return adata

# 设置日志
logging.basicConfig(level=logging.INFO)

# 数据和结果路径
DATA_DIR = "pancreas.h5ad"  # 修改为你的 pancreas.h5ad 文件路径
RES_PATH = "result"  # 结果保存路径，确保目录存在
data = "pancreas"
EXP_PARAMS = EXPERIMENT_INFO[data]
FILE_NAME = EXP_PARAMS["file_name"]

# 加载数据
logging.info(f"Loading dataset: {data}")
adata = sc.read("pancreas.h5ad")
condition_key = EXP_PARAMS["condition_key"]
cell_type_key = EXP_PARAMS["cell_type_key"]
reference = EXP_PARAMS["reference"]
query = EXP_PARAMS["query"]
print("Type of adata.X:", type(adata.X))
# 数据处理
import scipy.sparse
import scanpy as sc

def custom_remove_sparsity(adata):
    if scipy.sparse.issparse(adata.X):
        return sc.AnnData(X=adata.X.toarray(), obs=adata.obs.copy(deep=True), var=adata.var.copy(deep=True))
    return adata
adata = custom_remove_sparsity(adata)
source_adata = adata[adata.obs[condition_key].isin(reference)].copy()
target_adata = adata[adata.obs[condition_key].isin(query)].copy()
logging.info("Data loaded and processed successfully")
# 在 setup_anndata 之前
print("Batch categories:", source_adata.obs[condition_key].value_counts())
print("Label categories:", source_adata.obs[cell_type_key[0]].value_counts())
# 设置 anndata
SCVI.setup_anndata(source_adata, batch_key=condition_key, labels_key=cell_type_key[0])


# 训练参考模型
vae_ref = sca.models.SCVI(source_adata)
ref_time = time.time()
vae_ref.train()
vae_ref_scan = sca.models.SCANVI.from_scvi_model(vae_ref, unlabeled_category="Unknown")
vae_ref_scan.train(max_epochs=20)
ref_time = time.time() - ref_time
vae_ref_scan.save(f"{RES_PATH}/scanvi_model", overwrite=True)
logging.info("Reference model trained and saved")

# 训练查询模型
vae_q = sca.models.SCANVI.load_query_data(target_adata, f"{RES_PATH}/scanvi_model")
vae_q._unlabeled_indices = np.arange(target_adata.n_obs)
vae_q._labeled_indices = []
query_time = time.time()
vae_q.train(max_epochs=100, plan_kwargs=dict(weight_decay=0.0), check_val_every_n_epoch=10)
query_time = time.time() - query_time
vae_q.save(f"{RES_PATH}/scanvi_query_model", overwrite=True)
logging.info("Query model trained and saved")

# 评估整合指标
scores = None
try:
    print("target_adata shape:", target_adata.shape)
    print("source_adata shape:", source_adata.shape)
    adata_full = anndata.concat([target_adata, source_adata], axis=0, join="outer", label="query", keys=["Query", "Reference"])
    print("adata_full shape:", adata_full.shape)
    print("adata_full.obs columns:", adata_full.obs.columns)

    # 获取潜在表示
    adata_latent_full = sc.AnnData(vae_q.get_latent_representation(adata_full))
    adata_latent_full.obs["batch"] = adata_full.obs[condition_key].tolist()
    adata_latent_full.obs["celltype"] = adata_full.obs[cell_type_key[0]].tolist()

    # 集成指标计算
    conditions, _ = label_encoder(adata, condition_key=condition_key)
    labels, _ = label_encoder(adata, condition_key=cell_type_key[0])
    adata.obs["batch"] = conditions.squeeze(axis=1)
    adata.obs["celltype"] = labels.squeeze(axis=1)
    adata.obs["batch"] = adata.obs["batch"].astype("category")
    adata.obs["celltype"] = adata.obs["celltype"].astype("category")
    sc.pp.pca(adata)
    sc.pp.pca(adata_latent_full)

    adata.write(f"{RES_PATH}/adata_original.h5ad")
    adata_latent_full.write(f"{RES_PATH}/adata_latent.h5ad")

    scores = metrics(
        adata,
        adata_latent_full,
        "batch",
        "celltype",
        isolated_labels_asw_=True,
        silhouette_=True,
        graph_conn_=True,
        pcr_=True,
        isolated_labels_f1_=True,
        nmi_=True,
        ari_=True,
    )
    scores = scores.T
    scores = scores[
        [
            "NMI_cluster/label",
            "ARI_cluster/label",
            "ASW_label",
            "ASW_label/batch",
            "PCR_batch",
            "isolated_label_F1",
            "isolated_label_silhouette",
            "graph_conn",
        ]
    ]
    print("Integration Scores:\n", scores)
except Exception as e:
    print(f"Error in computing integration scores: {e}")

# 保存结果
results = {
    "reference_time": ref_time,
    "query_time": query_time,
    "integration_scores": scores,
}
logging.info("Results computed successfully")
print("\nResults:")
print("Reference Time (seconds):", results["reference_time"])
print("Query Time (seconds):", results["query_time"])
print("Integration Scores:\n", results["integration_scores"])

# 保存 integration_scores
if scores is not None:
    scores.to_csv(f"{RES_PATH}/integration_scores.csv")

 captum (see https://github.com/pytorch/captum).


Type of adata.X: <class 'scipy.sparse._csr.csr_matrix'>
Batch categories: study
inDrop3       3605
smartseq2     2394
inDrop1       1937
inDrop2       1724
smarter       1492
inDrop4       1303
fluidigmc1     638
Name: count, dtype: int64
Label categories: cell_type
alpha                 4459
beta                  3563
ductal                1557
acinar                1167
delta                  802
gamma                  571
activated_stellate     355
endothelial            287
quiescent_stellate     180
macrophage              63
mast                    35
epsilon                 27
schwann                 20
t_cell                   7
Name: count, dtype: int64


/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/cmml/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value

Epoch 1/400:   0%|          | 0/400 [00:00<?, ?it/s]

/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scvi/module/_vae.py:460: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scvi/module/_vae.py:460: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


...

/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scib/metrics/isolated_labels.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  batch_per_lab = tmp.groupby(label_key).agg({batch_key: "count"})


Cluster for iso_label_0.4 with leiden
Cluster for iso_label_0.6 with leiden
Cluster for iso_label_0.8 with leiden
Cluster for iso_label_1.0 with leiden
Cluster for iso_label_1.2 with leiden
Cluster for iso_label_1.4 with leiden
Cluster for iso_label_1.6 with leiden
Cluster for iso_label_1.8 with leiden
Cluster for iso_label_2.0 with leiden
Isolated labels ASW...


/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scib/metrics/isolated_labels.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  batch_per_lab = tmp.groupby(label_key).agg({batch_key: "count"})


Graph connectivity...
Integration Scores:
    NMI_cluster/label  ARI_cluster/label  ASW_label  ASW_label/batch  \
0           0.937513            0.96392   0.667629         0.850226   

   PCR_batch  isolated_label_F1  isolated_label_silhouette  graph_conn  
0   0.670202           0.148936                   0.720116    0.986879  

Results:
Reference Time (seconds): 551.3490998744965
Query Time (seconds): 65.85595846176147
Integration Scores:
    NMI_cluster/label  ARI_cluster/label  ASW_label  ASW_label/batch  \
0           0.937513            0.96392   0.667629         0.850226   

   PCR_batch  isolated_label_F1  isolated_label_silhouette  graph_conn  
0   0.670202           0.148936                   0.720116    0.986879  


/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/root/miniconda3/envs/cmml/lib/python3.11/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() i